In [1]:
import pandas as pd
import regex as re
import numpy as np

In [2]:
data = pd.read_csv("data/keywords.csv")

In [26]:
data["keywords_nan"] = data["keywords"].replace("[]", np.nan)

In [3]:
data.iloc[[0]].keywords[0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [27]:
def key_word_grabber(text):
    
    if text is not np.nan:
        
        return re.findall(r"'name':\s*'([^']*)'", text)
    else:
        return text

In [28]:
data["keywords_list"] = data["keywords_nan"].apply(lambda x: key_word_grabber(x))

In [31]:
data["keywords_nan"].isna().sum()

14795

In [32]:
data.shape

(46419, 4)

In [29]:
data["keywords_list"]

0        [jealousy, toy, boy, friendship, friends, riva...
1        [board game, disappearance, new home, recluse,...
2        [fishing, best friend, duringcreditsstinger, o...
3        [based on novel, interracial relationship, sin...
4        [baby, midlife crisis, confidence, aging, daug...
                               ...                        
46414                                        [tragic love]
46415                                [artist, play, pinoy]
46416                                                  NaN
46417                                                  NaN
46418                                                  NaN
Name: keywords_list, Length: 46419, dtype: object

In [6]:
data.shape

(46419, 3)

In [7]:
data["keywords"].value_counts()

[]                                                                                                                                                                                                                             14795
[{'id': 187056, 'name': 'woman director'}]                                                                                                                                                                                      1323
[{'id': 10183, 'name': 'independent film'}]                                                                                                                                                                                      509
[{'id': 9716, 'name': 'stand-up comedy'}]                                                                                                                                                                                        235
[{'id': 4344, 'name': 'musical'}]                                                   

In [8]:
type(data.iloc[46418]["keywords_list"])

list

In [9]:
data["keywords_list"].apply(lambda x: x)

0        [jealousy, toy, boy, friendship, friends, riva...
1        [board game, disappearance, new home, recluse,...
2        [fishing, best friend, duringcreditsstinger, o...
3        [based on novel, interracial relationship, sin...
4        [baby, midlife crisis, confidence, aging, daug...
                               ...                        
46414                                        [tragic love]
46415                                [artist, play, pinoy]
46416                                                   []
46417                                                   []
46418                                                   []
Name: keywords_list, Length: 46419, dtype: object

In [74]:
data2 = pd.read_csv("data/movies_metadata.csv")

/Users/nicolas/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
data2.drop(data2.loc[data2['overview']=="No overview found."].index, inplace=True)

In [37]:
data2.drop(data2.loc[data2['overview']=="No Overview"].index, inplace=True)

In [38]:
data2.drop(data2.loc[data2['overview']==""].index, inplace=True)

In [39]:
data2.drop(data2.loc[data2['overview']=="No movie overview available"].index, inplace=True)

In [63]:
def parse_overview(text):
    
    if text is not np.nan:
        
        if len(text) < 20:
            return np.nan
        else:
            return text
    

In [75]:
data2["overview_filtered"] = data2["overview"].apply(lambda x: parse_overview(x))

In [76]:
data2 = data2[data2["overview_filtered"].notna()]

In [86]:
data = data.drop(columns=["keywords_nan"])

In [94]:
data2["id"] = data2["id"].astype(int)

/var/folders/34/1yvhb8vj085g3m5tpn1x91d80000gn/T/ipykernel_72364/862761141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2["id"]=data2["id"].astype(int)


In [96]:
full_data = data2.merge(data, on="id", how="left")

In [71]:
data.sort_values(by="tmpp", key=lambda x: x.str.len()).head(30)

,id,keywords,keywords_list,keywords_nan,tmpp
21818,91745,"[{'id': 497, 'name': 'shakespeare'}, {'id': 13...","[shakespeare, teenager, middle ages, family fe...","[{'id': 497, 'name': 'shakespeare'}, {'id': 13...",Film by Risto Jarva.
20301,84328,[],NaN,NaN,American Documentary
35326,351819,"[{'id': 572, 'name': 'sex'}, {'id': 9215, 'nam...","[sex, black, parody, spoof, millionaire]","[{'id': 572, 'name': 'sex'}, {'id': 9215, 'nam...",Finnish documentary.
31468,87884,"[{'id': 293, 'name': 'female nudity'}, {'id': ...","[female nudity, teacher]","[{'id': 293, 'name': 'female nudity'}, {'id': ...",Based on true events
30334,13803,"[{'id': 12663, 'name': 'little girl'}, {'id': ...","[little girl, killer]","[{'id': 12663, 'name': 'little girl'}, {'id': ...",For a Book of Dollars
34664,337751,"[{'id': 9073, 'name': 'marilyn monroe'}, {'id'...","[marilyn monroe, woman director]","[{'id': 9073, 'name': 'marilyn monroe'}, {'id'...",Standup in Las Vegas.
43368,435351,"[{'id': 9716, 'name': 'stand-up comedy'}]",[stand-up comedy],"[{'id': 9716, 'name': 'stand-up comedy'}]",A journey into night.
22078,15991,"[{'id': 293, 'name': 'female nudity'}, {'id': ...","[female nudity, orgasm]","[{'id': 293, 'name': 'female nudity'}, {'id': ...",A film by Jari Halonen
27491,171292,"[{'id': 174878, 'name': 'female vampire'}]",[female vampire],"[{'id': 174878, 'name': 'female vampire'}]",A man dances on curbs.
43931,255552,"[{'id': 380, 'name': 'brother brother relation...","[brother brother relationship, family, missing...","[{'id': 380, 'name': 'brother brother relation...",Love on the front page


In [12]:
import spacy

In [13]:
#!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [99]:
nlp = spacy.load('en_core_web_md')

In [114]:
valid_POS = set(['VERB', 'NOUN', 'ADJ', 'PROPN'])
specific_stw = set(['relevant', 'simple', 'base'])

def text_preprocessing(rawtext):
    #Sentence tokenization and filtering of non-English sentences
    #mult_doc = mult_nlp(rawtext)
    #english_text = ' '.join([sent.text for sent in mult_doc.sents if sent._.language['language']=='en'])

    #Tokenization
    english_doc = nlp(rawtext)

    new_tokens = list()

    for token in english_doc:
        # POS
        if token.pos_ not in valid_POS:
            continue
        
        # alphanumeric
        if not token.is_alpha:
            continue

        # stopwords
        if token.is_stop or token.text in specific_stw:
            continue

        new_tokens.append(token)

    # Lemmatization
    lemmatized_text = ' '.join([token.lemma_ for token in new_tokens])
    return lemmatized_text

In [115]:
full_data["preprocessed_overview"] = full_data["overview_filtered"].apply(lambda x: text_preprocessing(x))

In [116]:
corpus = [row['preprocessed_overview'].split() for idx, row in full_data.iterrows()]

In [117]:
from gensim.corpora import Dictionary

# Create dictionary of tokens
D = Dictionary(corpus)

In [119]:
reviews_bow = [D.doc2bow(doc) for doc in corpus]

n_review = 1000
print('============= Review (lemmas) =============')
print(' '.join(corpus[n_review]))

print('\n============= Sparse vector representation =============')
print(reviews_bow[n_review])

print('\n============= Word counts for the review =============')
print(list(map(lambda x: (D[x[0]], x[1]), reviews_bow[n_review])))

============= Review (lemmas) =============
Pennsylvania band score hit ride star make machinery lot help manager

============= Sparse vector representation =============
[(299, 1), (633, 1), (1022, 1), (1031, 1), (1053, 1), (1451, 1), (1452, 1), (2335, 1), (3171, 1), (5937, 1), (7998, 1)]

============= Word counts for the review =============
[('star', 1), ('help', 1), ('band', 1), ('hit', 1), ('make', 1), ('lot', 1), ('manager', 1), ('ride', 1), ('Pennsylvania', 1), ('score', 1), ('machinery', 1)]


In [120]:
from gensim.models import TfidfModel

tfidf = TfidfModel(reviews_bow)

In [121]:
reviews_tfidf = tfidf[reviews_bow]
n_project = 1000
print('============= TFIDF representation for the project =============')
print(reviews_tfidf[n_review])

print('\n============= TFIDF applying the transformation only to the document =============')
print(tfidf[reviews_bow[n_review]])

============= TFIDF representation for the project =============
[(299, 0.195012524874713), (633, 0.16796274707208164), (1022, 0.25016603620399097), (1031, 0.2500826419276045), (1053, 0.20458547875315558), (1451, 0.2895699753904568), (1452, 0.3041608918825714), (2335, 0.28024346803581934), (3171, 0.41318113138755014), (5937, 0.33150713897989326), (7998, 0.480897482269683)]

============= TFIDF applying the transformation only to the document =============
[(299, 0.195012524874713), (633, 0.16796274707208164), (1022, 0.25016603620399097), (1031, 0.2500826419276045), (1053, 0.20458547875315558), (1451, 0.2895699753904568), (1452, 0.3041608918825714), (2335, 0.28024346803581934), (3171, 0.41318113138755014), (5937, 0.33150713897989326), (7998, 0.480897482269683)]


In [122]:
reviews_tfidf

In [127]:
full_data["preprocessed_overview"]

0        lead Woody Andy toy live room Andy birthday br...
1        sibling Judy Peter discover enchanted board ga...
2        family wedding reignite ancient feud door neig...
3        cheat mistreat step woman hold breath wait elu...
4        George Banks recover daughter wedding receive ...
                               ...                        
45291                                  rise fall man woman
45292    artist struggle finish work storyline cult pla...
45293    hit go wrong professional assassin end suitcas...
45294    small town live brother minister hunchback pai...
45295    year decriminalisation homosexuality UK direct...
Name: preprocessed_overview, Length: 45296, dtype: object

In [147]:
" ".join(full_data.iloc[0]["keywords_list"])

'jealousy toy boy friendship friends rivalry boy next door new toy toy comes to life'

In [150]:
full_data["keywords_list"] = full_data["keywords_list"].apply(lambda x: " ".join(x) if x is not np.nan else x)

In [154]:
full_data["preprocessed_keywords"] = full_data["keywords_list"].apply(lambda x: text_preprocessing(x) if x is not np.nan else x)

In [155]:
full_data["preprocessed_keywords"]

0        jealousy toy boy friendship friend rivalry boy...
1        board game disappearance new home recluse gian...
2            fish good friend duringcreditsstinger old man
3        base novel interracial relationship single mot...
4        baby midlife crisis confidence age daughter mo...
                               ...                        
45291                                          tragic love
45292                                    artist play pinoy
45293                                                  NaN
45294                                                  NaN
45295                                                  NaN
Name: preprocessed_keywords, Length: 45296, dtype: object

In [152]:
full_data["overview_plus_keyword_preprocessed"] = full_data["preprocessed_overview"] + full_data["preprocessed_keywords"]

In [136]:
# create feature and target dataframes
X = full_data[["preprocessed_overview"]]
y = full_data["vote_average"]

In [137]:
from sklearn.model_selection import train_test_split

# create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, 
                                                    random_state = 21, shuffle = True)

In [138]:
X_train

,preprocessed_overview
35228,year old cycling talent Freddy son butcher idy...
13737,overweight loner Rino spend day seclusion dad ...
34687,Raggedy Ann Andy leave playroom rescue Babette...
30067,Zenon Kar teenager live space station year com...
32779,orphan young age Scriptwriter Amit military pi...
...,...
16432,Caitlin Fairchild teenager offer place institu...
8964,busload woman strand isolated canadian country...
5944,Monty Python live Hollywood Bowl concert film ...
5327,beautiful sophisticated woman Oscar Grubman se...


In [139]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence 

# sequence words
# X_train_seq is a list with each element being a list of each word in one article as a separate string

X_train_seq = [text_to_word_sequence(zz) for zz in X_train["preprocessed_overview"]]
X_test_seq = [text_to_word_sequence(zz) for zz in X_test["preprocessed_overview"]]

In [161]:
from gensim.models import Doc2Vec

In [165]:
%%time
from gensim.models import Doc2Vec

# dimensions of vector embedding
dim_embedding = 100

# create Word2Vec on Train set
d2v = Doc2Vec(sentences = X_train_seq, size = dim_embedding, window = 5, min_count = 1)

/Users/nicolas/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/gensim/models/doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


AttributeError: 'list' object has no attribute 'words'

In [164]:
d2v

NameError: name 'd2v' is not defined

In [163]:
d2v.wv.most_similar("thriller")

NameError: name 'd2v' is not defined